In [170]:
""" Este script extrae información de los campos del HTML 
    del cvlac a partir de una base de datos inicial de los perfiles
"""
# Importar librerias/Modulos
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
import time

In [171]:
#parser = pag_principal()

def links_pag_principal():

    link = 'https://www.argentina.gob.ar/ciencia/financiamiento'
    encabezados = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
        }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text    

    response = scrapy.Selector(text=resp)
    links = response.xpath('//div[@class="col-xs-12 col-sm-6 col-md-6"]/a/@href').getall()

    urls = []
    url = 'https://www.argentina.gob.ar'
    for link in links:
        urls.append(url+link)


    return urls


def links_proyectos(link):
    #link = 'https://www.argentina.gob.ar/ciencia/financiamiento'

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    response = scrapy.Selector(text=resp)
    links = response.xpath('//nav/ul/li[@class=""]/a/@href').getall()

    urls = []
    url = 'https://www.argentina.gob.ar'
    for link in links:
        urls.append(url+link)


    return urls

def parser_proyecto(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    parser = html.fromstring(resp)

    return parser


def titulo(parser):
    titulo_proyecto = parser.xpath('//h1[@class="activities-sidbar"]//text()')
    return titulo_proyecto


def descripcion(parser):

    descripción = parser.xpath('//div[@class=""]//text()')

    text = ''
    for i in descripción:
        text = text+i
    
    return text

def fecha_cierre(parser):
    try:
        fecha_cierre= parser.xpath('//div[@class="media-body"]//text()')
        if fecha_cierre[0] != 'Fecha de cierre':
            fecha_cierre = 'Ventanilla permanente'
        else:
            fecha_cierre = fecha_cierre[1]

    except IndexError:
        fecha_cierre=''

    return fecha_cierre


def pdf(link):

    encabezados = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
        }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    response = scrapy.Selector(text=resp)
    links = response.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//@href').getall()
    urls=''
    if len(links) == 0:
        urls = ''
    else:
        for link in links:
            urls = urls+link+ ', '

    return urls.rstrip(', ')

def links_cerradas_segundo_nivel(link):

    encabezados = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
        }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    response = scrapy.Selector(text=resp)
    links = response.xpath('//ul/li[@class="active"]//li/a/@href').getall()

    url = 'https://www.argentina.gob.ar'
    urls = []
    for link in links:
        urls.append(url + link)

    return urls

def descripcion_vertical(parser):

    try:
        links = parser.xpath('//div/p/a/@href')[-1]

        parser = parser_proyecto(links)
        contenidos = parser.xpath('//div[@class="field-item even"]//p/text()')

        contenido = ''
        for content in contenidos:
            contenido = contenido+content
    except:
        contenido = ''


    return contenido

def pdf_vertical(parser):
    try:
        links = parser.xpath('//div/p/a/@href')[-1]

        parser2 = parser_proyecto(links)
        links = parser2.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//a/@href')
        if links == '':
            links = parser2.xpath('//section/h3[1]//a/@href')
        urls = ''
        for url in links:
            urls = urls + url + ', '
    except:
        links = parser.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//a/@href')
        if links == '':
            links = parser.xpath('//section/h3[1]//a/@href')
        urls = ''
        for url in links:
            urls = urls + url + ', '

    
    return urls.rstrip(', ')
    



In [172]:
# abiertas
urls = links_pag_principal()

links_abiertas = links_proyectos(urls[0])
#parser_cerradas = parser(urls[1])
argentina = pd.DataFrame()
titulos = []
descripciones = []
fechas_cierre = []
pdfs = []


for link in links_abiertas:
   
    parser = parser_proyecto(link)
    
    titulos.append(titulo(parser)[0])
    descripciones.append(descripcion(parser) + '---' + descripcion_vertical(parser))
    fechas_cierre.append(fecha_cierre(parser))

    if pdf(link) == '':
        pdfs.append(pdf_vertical(parser))
    else:
        pdfs.append(pdf(link))
    

argentina['Tútulo'] = titulos
argentina['Descripción'] = descripciones
argentina['Fecha Cierre'] = fechas_cierre
argentina['Links pdf'] = pdfs
argentina['Ano'] = 2021
argentina['Estado de la convocatoria'] = 'Convocatoria Abierta'
argentina['Link'] = links_abiertas


In [173]:
####
#### Cerradas
####

urls = links_pag_principal()

links_cerradas = links_proyectos(urls[1])
for link_cerrada in links_cerradas:
    
    links_segundo_nivel = links_cerradas_segundo_nivel(str(link_cerrada))
    cerradas = pd.DataFrame()
    
    titulos = []
    descripciones = []
    fechas_cierre = []
    pdfs = []

    for segundo_nivel in links_segundo_nivel:
        parser = parser_proyecto(segundo_nivel)

        titulos.append(titulo(parser)[0])
        descripciones.append(descripcion(parser) + '---' + descripcion_vertical(parser))
        fechas_cierre.append(fecha_cierre(parser))

        if pdf(link) == '':
            pdfs.append(pdf_vertical(parser))
        else:
            pdfs.append(pdf(link))

    year = link_cerrada.split('-')[-1]
    print(year)
    cerradas['Tútulo'] = titulos
    cerradas['Descripción'] = descripciones
    cerradas['Fecha Cierre'] = fechas_cierre
    cerradas['Links pdf'] = pdfs
    cerradas['Ano'] = year
    cerradas['Estado de la convocatoria'] = 'Convocatoria Cerrada'
    cerradas['Link'] = links_segundo_nivel

    argentina = argentina.append(cerradas)

argentina['País'] = 'Argentina'
argentina['Objetivo'] = ''
argentina['Cuantia'] = ''
argentina['Fecha Apertura'] = ''
argentina['Fecha Resultados Preliminares'] = ''   
argentina['Fecha Publicación Resultados Definitivos'] = ''   
argentina['Público Objetivo'] = ''


2021
2020
2019
2018
2017
2016
2015
2014
2013


In [174]:
argentina

,Tútulo,Descripción,Fecha Cierre,Links pdf,Ano,Estado de la convocatoria,Link,País,Objetivo,Cuantia,Fecha Apertura,Fecha Resultados Preliminares,Fecha Publicación Resultados Definitivos,Público Objetivo
0,Premios RAICES y Leloir,Los premios RAICES están destinados a científi...,Ventanilla permanente,https://www.argentina.gob.ar/sites/default/fil...,2021,Convocatoria Abierta,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
1,Distinción Argentino-Israelí en Innovación 2020,"El Ministerio de Ciencia, Tecnología e Innovac...",9 de abril de 2021,https://www.argentina.gob.ar/sites/default/fil...,2021,Convocatoria Abierta,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
2,Concurso Innova Salud,"El Ministerio de Ciencia, Tecnología e Innovac...",11 de marzo de 2021,https://www.argentina.gob.ar/sites/default/fil...,2021,Convocatoria Abierta,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
3,Subsidios de retorno,Los Subsidios de retorno tienen el propósito d...,Ventanilla permanente,https://www.argentina.gob.ar/sites/default/fil...,2021,Convocatoria Abierta,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
4,Programa “ImpaCT.AR Ciencia y Tecnología”,El programa ImpaCT.AR tiene el propósito de pr...,Ventanilla permanente,https://www.argentina.gob.ar/sites/default/fil...,2021,Convocatoria Abierta,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Apoyo tecnológico al sector turismo 2017,El objetivo de la convocatoria es dar impulso ...,21 de febrero de 2018 a las 12.00 PM,https://www.argentina.gob.ar/sites/default/fil...,2017,Convocatoria Cerrada,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
1,Concurso Emprendedor Tecnológico,Cien proyectos/startups accedieron a un proces...,13 de julio de 2017 18.00 horas,https://www.argentina.gob.ar/sites/default/fil...,2017,Convocatoria Cerrada,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
2,Innovación + impacto social,Se convocó a proyectos de emprendedores social...,10 de noviembre de 2017,https://www.argentina.gob.ar/sites/default/fil...,2017,Convocatoria Cerrada,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,
0,Programa de cooperación MinCyT - CRG 2014,"Sobre la convocatoriaEl Ministerio de Ciencia,...",16 de mayo de 2014,,2014,Convocatoria Cerrada,https://www.argentina.gob.ar/ciencia/financiam...,Argentina,,,,,,


In [176]:
argentina.to_excel('Argentina.xlsx')